In [12]:
import os
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('192.168.0.209', 27017)
db = client['capstone_1']
cards = db['cards']
decks = db['deck_lists']

# Base URLs for building requests
scryfall_api_url = 'https://api.scryfall.com/{}'
mtgtop8_url = 'https://www.mtgtop8.com/{}'

# mtgtop8.com format keys for building requests
mt8_format_keys = {
    'vintage': 'VI',
    'legacy': 'LE',
    'modern': 'MO',
    'pioneer': 'PI',
    'historic': 'HI',
    'standard': 'ST',
    'commander': 'EDH',
    'limited': 'LI',
    'pauper': 'PAU',
    'peasant': 'PEA',
    'block': 'BL',
    'extended': 'EX',
    'highlander': 'HIGH',
    'canadian_highlander': 'CHL'
}

def query(link, payload={}):
    """A requests wrapper function"""
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print('WARNING', response.status_code)
        print(response.content)
    return response

def hot_soup(url, payload={}):
    """Cooks a steaming bowl of hot soup"""
    response = query(url, payload)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def get_card(name_str):
    """Returns data from Scryfall API on an individual card by name"""
    payload = {'fuzzy': '+'.join(name_str.split())}
    response = query(scryfall_api_url.format('cards/named'), payload)
    return response.json()

I originally inteded to scrape the official WOTC magic.wizards.com site for tournament results but they unfortunately don't make much available outside of the most recent events.
After some research, tournament data will be scraped from mtgtop8.com, who offer *years* worth of back data.
Some individual card data could be scraped here as well, but scryfall.com offers much more robust card data via a convenient API.

interesting note on data gathering: https://www.dailyesports.gg/wizards-of-the-coast-mtg-frank-karsten-stop-publishing-gp-results-win-rates/

In [16]:
soup = hot_soup(mtgtop8_url.format('format'), {'f': mt8_format_keys['standard']})

In [24]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)
test = driver.get('https://www.google.com')
